<a href="https://colab.research.google.com/github/srijit43/NLP-theory-and-code/blob/main/03_03_Ngram_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectorizing Raw Data: N-Grams

### N-Grams

Creates a document-term matrix where counts still occupy the cell but instead of the columns representing single terms, they represent all combinations of adjacent words of length n in your text.

"NLP is an interesting topic"

| n | Name      | Tokens                                                         |
|---|-----------|----------------------------------------------------------------|
| 2 | bigram    | ["nlp is", "is an", "an interesting", "interesting topic"]      |
| 3 | trigram   | ["nlp is an", "is an interesting", "an interesting topic"] |
| 4 | four-gram | ["nlp is an interesting", "is an interesting topic"]    |

### Read in text

In [1]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Create function to remove punctuation, tokenize, remove stopwords, and stem

In [2]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])  #joining individual characters together to return words to tokenize
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords]) #return the sentence joining the words after blank spaces hence the " "
    return text

data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text(x))
data.head()

,label,body_text,cleaned_text
0,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri questionstd...
1,ham,"Nah I don't think he goes to usf, he lives around here though",nah dont think goe usf live around though
2,ham,Even my brother is not like to speak with me. They treat me like aids patent.,even brother like speak treat like aid patent
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,date sunday
4,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your call...,per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend...


### Apply CountVectorizer (w/ N-Grams)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_vect = CountVectorizer(ngram_range=(2,2))  #we are only looking for bigrams. (1,3) --> indicates uni, bi and tri

X_counts = n_gram_vect.fit_transform(data['cleaned_text'])

print(X_counts.shape)  #31000 unique combos of 2 words

print(n_gram_vect.get_feature_names_out())



(5567, 31260)
['008704050406 sp' '0089mi last' '0121 2025050' ... 'üll submit'
 'üll take' '〨ud even']


### Apply CountVectorizer (w/ N-Grams) to smaller sample

In [7]:
data_sample = data[0:20]

Applying the method on a smaller data sample

In [12]:
#Code for smaller data set
n_gram_vect_sample = CountVectorizer(ngram_range=(2,2))

X_counts_sample = n_gram_vect_sample.fit_transform(data_sample['cleaned_text'])

print(X_counts_sample.shape)

print(n_gram_vect_sample.get_feature_names_out())


(20, 198)
['09061701461 claim' '100 20000' '100000 prize' '11 month' '12 hour'
 '150pday 6day' '16 tsandc' '20000 pound' '2005 text' '21st may'
 '4txtú120 poboxox36504w45wq' '6day 16' '81010 tc' '87077 eg'
 '87077 trywal' '87121 receiv' '87575 cost' '900 prize' 'aft finish'
 'aid patent' 'alright way' 'anymor tonight' 'appli 08452810075over18'
 'appli repli' 'ard smth' 'around though' 'brother like'
 'call 09061701461' 'call mobil' 'caller press' 'callertun caller'
 'camera free' 'cash 100' 'chanc win' 'claim 81010' 'claim call'
 'claim code' 'click httpwap' 'click wap' 'co free' 'code kl341'
 'colour mobil' 'comp win' 'copi friend' 'cost 150pday' 'credit click'
 'cri enough' 'csh11 send' 'cup final' 'custom select' 'da stock'
 'date sunday' 'dont miss' 'dont think' 'dont want' 'eg england'
 'eh rememb' 'england 87077' 'england macedonia' 'enough today'
 'entitl updat' 'entri questionstd' 'entri wkli' 'even brother' 'fa 87121'
 'fa cup' 'feel way' 'ffffffffff alright' 'final tkt' 'fine

### Vectorizers output sparse matrices

_**Sparse Matrix**: A matrix in which most entries are 0. In the interest of efficient storage, a sparse matrix will be stored by only storing the locations of the non-zero elements._

In [15]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())
X_counts_df.columns = n_gram_vect_sample.get_feature_names_out()
print(X_counts_df)

    09061701461 claim  100 20000  100000 prize  11 month  12 hour  \
0                   0          0             0         0        0   
1                   0          0             0         0        0   
2                   0          0             0         0        0   
3                   0          0             0         0        0   
4                   0          0             0         0        0   
5                   1          0             0         0        1   
6                   0          0             0         1        0   
7                   0          0             0         0        0   
8                   0          1             0         0        0   
9                   0          0             1         0        0   
10                  0          0             0         0        0   
11                  0          0             0         0        0   
12                  0          0             0         0        0   
13                  0          0  